In [1]:
from graphdatascience import GraphDataScience

host = "bolt://localhost:7687"
user = "neo4j"
password = "letmein"

gds = GraphDataScience(host, auth=(user, password))

In [2]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE;
"""
)

""


In [3]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (k:Keyword) REQUIRE k.name IS UNIQUE;
"""
)

""


In [ ]:
# importing the module
import json

# Opening JSON file
with open("neo4j_docs.json") as json_file:
    data = json.load(json_file)

    # Print the type of data variable
    print("Type:", type(data))

In [ ]:
import_query = """

UNWIND $data AS row
MERGE (p:Page {url:row.url})
SET p.embedding = row.embedding,
    p.has_text = row.has_text
FOREACH (l in row.links    | MERGE (p1:Page {url:l}) MERGE (p)-[:LINKS_TO]->(p1))
FOREACH (k in row.keywords | MERGE (k1:Keyword {name:k}) MERGE (p)-[:HAS_KEYWORD]->(k1))
"""
x = 1
params = []
for key in data:
    params.append(
        {
            "url": key,
            "embedding": data[key]["embeddings"],
            "keywords": data[key]["keywords"],
            "links": data[key]["links"],
            "has_text": True if data[key]["text"] else False,
        }
    )
    # Batch per 500
    if len(params) == 500:
        gds.run_cypher(import_query, {"data": params})
        params = []
        # Logging
        print(f"Importing {x} batch")
        x += 1

# Import the remainder
gds.run_cypher(import_query, {"data": params})